In [4]:
# for validation
import numpy as np
import pandas as pd
import datetime
from netCDF4 import Dataset,num2date,date2num
import gpflow as gf
from eofs.standard import Eof
import time
from datetime import datetime, timedelta
import numpy.ma as ma
import os
from typing import Sequence
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [5]:
# Load historical simulation data
historical_simulation_data = Dataset('D:\CWatM-new\Brahmaputra\output229\discharge_daily_mpi.nc','r')

# Load historical observation data
historical_observation_data = Dataset('E:\data\era5_runoff\disdata\era5_data1979-2014_5min.nc','r')

mpi_hist_train_origin = historical_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

obsdata_origin = historical_observation_data.variables['dis24'][:]
historical_observation_data.close()
historical_simulation_data.close()

mask_t = mpi_hist_train_origin == mpi_hist_train_origin[1][0][0]
mpi_hist_train = ma.array(mpi_hist_train_origin,mask = mask_t)
obsdata = ma.array(obsdata_origin,mask = mask_t)
#数据切片调试-可以不做
#mpi_hist_train_old = ma.array(mpi_hist_train_origin,mask = mask_t)
#obsdata_old = ma.array(obsdata_origin,mask = mask_t)
#mpi_hist_train = mpi_hist_train_old[13000:][:][:]
#obsdata = obsdata_old[13000:][:][:]

In [6]:
#提取两个数据的主成分，减少分析两个数据差异的复杂度
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
#coslat = np.cos(np.deg2rad(lats)).clip(0., 1.)
#wgts = np.sqrt(coslat)[..., np.newaxis]


# Perform EOF analysis on data1 and data2
#obsdata_solver = Eof(obsdata, weights=None, center=False)
obsdata_solver = Eof(obsdata, weights=None,center=False)

#explain for how much variance
obsdata_mode = obsdata_solver.varianceFraction(neigs=4)
print(obsdata_mode)

# Retrieve the leading EOFs for both datasets
obsdata_eofs = obsdata_solver.eofs(neofs=4)
print(obsdata_eofs.shape)

#variance correlation
obsdata_eofc = obsdata_solver.eofsAsCorrelation(neofs=4)


#create pseudo-ECs for GPR

#源代码中obsdata 不存在 obsdata -> obsdata_old
mpi_hist_train_ECs = obsdata_solver.projectField(mpi_hist_train,neofs = 4)
obsdata_ECs = obsdata_solver.projectField(obsdata,neofs = 4)
print(mpi_hist_train_ECs.shape)

[0.9868846  0.0065986  0.00354106 0.00170928]
(4, 89, 189)
(13149, 4)


In [4]:
#将两列ECs各自做为训练集
mpi_hist_train_ECs_l1 = mpi_hist_train_ECs[:,0].reshape(13149,1)
mpi_hist_train_ECs_x1 = tf.convert_to_tensor(mpi_hist_train_ECs_l1)
mpi_hist_train_ECs_x1=tf.cast(mpi_hist_train_ECs_x1,tf.float64)
mpi_hist_train_ECs_l2 = mpi_hist_train_ECs[:,1].reshape(13149,1)
mpi_hist_train_ECs_x2 = tf.convert_to_tensor(mpi_hist_train_ECs_l2)
mpi_hist_train_ECs_x2=tf.cast(mpi_hist_train_ECs_x2,tf.float64)
mpi_hist_train_ECs_l3 = mpi_hist_train_ECs[:,2].reshape(13149,1)
mpi_hist_train_ECs_x3 = tf.convert_to_tensor(mpi_hist_train_ECs_l3)
mpi_hist_train_ECs_x3=tf.cast(mpi_hist_train_ECs_x3,tf.float64)
mpi_hist_train_ECs_l4 = mpi_hist_train_ECs[:,3].reshape(13149,1)
mpi_hist_train_ECs_x4 = tf.convert_to_tensor(mpi_hist_train_ECs_l4)
mpi_hist_train_ECs_x4=tf.cast(mpi_hist_train_ECs_x4,tf.float64)
obsdata_ECs_l1 = obsdata_ECs[:,0].reshape(13149,1)
obsdata_ECs_y1 = tf.convert_to_tensor(obsdata_ECs_l1)
obsdata_ECs_y1=tf.cast(obsdata_ECs_y1,tf.float64)
obsdata_ECs_l2 = obsdata_ECs[:,1].reshape(13149,1)
obsdata_ECs_y2 = tf.convert_to_tensor(obsdata_ECs_l2)
obsdata_ECs_y2=tf.cast(obsdata_ECs_y2,tf.float64)
obsdata_ECs_l3 = obsdata_ECs[:,2].reshape(13149,1)
obsdata_ECs_y3 = tf.convert_to_tensor(obsdata_ECs_l3)
obsdata_ECs_y3=tf.cast(obsdata_ECs_y3,tf.float64)
obsdata_ECs_l4 = obsdata_ECs[:,3].reshape(13149,1)
obsdata_ECs_y4 = tf.convert_to_tensor(obsdata_ECs_l4)
obsdata_ECs_y4=tf.cast(obsdata_ECs_y4,tf.float64)

In [5]:
#EOF1
#standardize x,y
n_input_ECs = 4
scaler_x = StandardScaler()
scaler_y = StandardScaler()
mpi_train = np.c_[mpi_hist_train_ECs_x1,mpi_hist_train_ECs_x2,mpi_hist_train_ECs_x3,mpi_hist_train_ECs_x4]
mpi_train_sc = scaler_x.fit_transform(mpi_train)
#inducing（修改）
n_inducing_points=700
model_mpi_eof = list()
obs_train_all = np.c_[obsdata_ECs_y1,obsdata_ECs_y2,obsdata_ECs_y3,obsdata_ECs_y4]
for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mpi_train_sc[:,0].min(), mpi_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(mpi_train_sc[:, j].min(), mpi_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mpi_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mpi_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mpi_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_mpi_eof[i].training_loss, model_mpi_eof[i].trainable_variables)
    gf.utilities.print_summary(model_mpi_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_mpi_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_mpi_eof1_timeperiod:',opt_mpi_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_mpi_eof[i].predict_y(mpi_train_sc, full_cov=False)
    mpi_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mpi_pred_mean) / np.mean(obs_train)
    y = (np.std(mpi_pred_mean) / np.mean(mpi_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.00241
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.94623
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.21739
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.71668, -1.50329, -4.05911..."


The  1  ECs training for obsdata
opt_mpi_eof1_timeperiod: 46456.29700000002
0.9993125252720713 0.8809427175483832


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.7224
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.65789
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.708
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.77844, -1.77264, -4.18731..."


The  2  ECs training for obsdata
opt_mpi_eof1_timeperiod: 46871.96899999998
1.13492668566526 0.44641729849487527


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.31846
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.54808
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.86083
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.9063, -2.09145, -4.48771..."


The  3  ECs training for obsdata
opt_mpi_eof1_timeperiod: 49611.84400000004
1.043537319584797 0.30751324719067424


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,2.25982
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,5.71267
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.91595
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.89081, -2.06246, -4.52679..."


The  4  ECs training for obsdata
opt_mpi_eof1_timeperiod: 95.82799999997951
1.0010200684882973 0.35266416074727097


In [6]:
mpi_pred_ECs = np.zeros((13149,n_input_ECs))
mpi_pred_ECs_lb = np.zeros((13149,n_input_ECs))
mpi_pred_ECs_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

for i in range(n_input_ECs):
    y_pred_mean, y_pred_var= model_mpi_eof[i].predict_y(mpi_train_sc, full_cov=False)
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    print(y_pred_mean,obs_train_sc)
    mpi_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    print(mpi_pred_mean,obs_train)
    B = np.mean(mpi_pred_mean) / np.mean(obs_train)
    y = (np.std(mpi_pred_mean) / np.mean(mpi_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)
    mpi_pred_ECs[:,i] =np.squeeze(mpi_pred_mean)
    print(mpi_pred_ECs[:,i])
    y__pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    mpi_pred_ECs_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    mpi_pred_ECs_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
mpi_hist_train_ECs_SGPRcor=tf.concat([mpi_pred_ECs[:,0].reshape(13149, 1),mpi_pred_ECs[:,1].reshape(13149, 1),mpi_pred_ECs[:,2].reshape(13149, 1),mpi_pred_ECs[:,3].reshape(13149, 1)],axis=1)
mpi_hist_train_ECs_lb_SGPRcor=tf.concat([mpi_pred_ECs_lb[:,0].reshape(13149, 1),mpi_pred_ECs_lb[:,1].reshape(13149, 1),mpi_pred_ECs_lb[:,2].reshape(13149, 1),mpi_pred_ECs_lb[:,3].reshape(13149, 1)],axis=1)
mpi_hist_train_ECs_ub_SGPRcor=tf.concat([mpi_pred_ECs_ub[:,0].reshape(13149, 1),mpi_pred_ECs_ub[:,1].reshape(13149, 1),mpi_pred_ECs_ub[:,2].reshape(13149, 1),mpi_pred_ECs_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(mpi_hist_train_ECs_SGPRcor,columns = ['mpi_EC1','mpi_EC2','mpi_EC3','mpi_EC4'])
df_lb_merge = pd.DataFrame(mpi_hist_train_ECs_lb_SGPRcor,columns = ['mpi_EC1_lb','mpi_EC2_lb','mpi_EC3_lb','mpi_EC4_lb'])
df_ub_merge = pd.DataFrame(mpi_hist_train_ECs_ub_SGPRcor,columns = ['mpi_EC1_ub','mpi_EC2_ub','mpi_EC3_ub','mpi_EC4_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

tf.Tensor(
[[0.82554387]
 [0.85891331]
 [0.88170956]
 ...
 [0.90518633]
 [0.90472923]
 [0.90336269]], shape=(13149, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[[-41938.81313029]
 [-37293.73450899]
 [-34120.46005177]
 ...
 [-30852.45557246]
 [-30916.08455279]
 [-31106.30934385]] [[-26780.41015625]
 [-26305.3046875 ]
 [-25864.76757812]
 ...
 [-31726.390625  ]
 [-31462.87109375]
 [-31136.0703125 ]]
0.9993125252720713 0.8809427175483832
[-41938.81313029 -37293.73450899 -34120.46005177 ... -30852.45557246
 -30916.08455279 -31106.30934385]
tf.Tensor(
[[ 0.09486535]
 [ 0.03130418]
 [-0.01927986]
 ...
 [ 0.01519553]
 [ 0.0194042 ]
 [ 0.02456547]], shape=(13149, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[[1253.09120036]
 [ 163.37529142]
 [-703.85605968]
 ...
 [-112.79735525]
 [ -40.64241851]
 [  47.84428254]] [[-2039.22070312]
 [-1982.69091797]
 [-1931.41503

In [8]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
mpi_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
mpi_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
mpi_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_mpi_eof_2GP = list()
mpi_pred_ECs_range = np.zeros((13149,n_input_ECs))
for i in range(n_input_ECs):
    mpi_pred_ECs_range[:,i] = np.subtract(mpi_pred_ECs_ub[:,i],mpi_pred_ECs_lb[:,i])
    mpi_train_2GP = np.c_[mpi_pred_ECs[:,i],mpi_pred_ECs_lb[:,i],mpi_pred_ECs_ub[:,i],mpi_pred_ECs_range[:,i] ]
    mpi_train_2GP_sc = scaler_z.fit_transform(mpi_train_2GP)
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mpi_train_2GP_sc[:,0].min(), mpi_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(mpi_train_2GP_sc[:, j].min(), mpi_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mpi_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mpi_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mpi_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_mpi_eof_2GP[i].training_loss, model_mpi_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_mpi_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_mpi_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_mpi_eof1_2GP_timeperiod:',opt_mpi_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_mpi_eof_2GP[i].predict_y(mpi_train_2GP_sc, full_cov=False)
    mpi_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mpi_pred_mean) / np.mean(obs_train)
    y = (np.std(mpi_pred_mean) / np.mean(mpi_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    mpi_pred_ECs_2GP[:,i] =np.squeeze(mpi_pred_mean)
    print(mpi_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    mpi_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    mpi_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,200.90294
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,44.52664
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.20926
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-2.48691, -2.62419, -3.04979..."


The  1  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 95.82799999997951
[-42394.87406204 -37926.21915824 -34894.90579677 ... -31795.61465447
 -31855.53436444 -32037.35978937]
0.9999987634806988 0.8891886183259272


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,2.79595
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,9.0399
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.66449
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.72114, -3.74812, -3.81074..."


The  2  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 95.82799999997951
[1458.81297851  276.87590903 -673.66693338 ...  -33.1390414    45.70999233
  142.20010718]
0.9995082834641598 0.5791796336664786


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,27.10141
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,15.23191
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.80619
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.74171, -5.24404, -5.61097..."


The  3  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 95.82799999997951
[2416.49333362 2353.03732658 2381.30784097 ... 2353.76644816 2340.55376777
 2325.61501079]
1.00002038644662 0.4398357683025285


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.4379
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,1.48947
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.81949
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.35024, -4.3313, -4.32476..."


The  4  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 95.82799999997951
[1110.63848977  720.46658151  570.22224737 ...  381.92557709  381.72508584
  381.974223  ]
1.0065471341539909 0.419081950338638


In [9]:
#mpi_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
#mpi_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

#for i in range(n_input_ECs):
#    y_pred_mean, y_pred_var= model_mpi_eof_2GP[i].predict_y(mpi_train_2GP_sc, full_cov=False)
#    obs_train = obs_train_all[:,i].reshape(13149,1)
#    obs_train_sc = scaler_y.fit_transform(obs_train)
#    print(y_pred_mean,obs_train_sc)
#    mpi_pred_mean_2GP = scaler_y.inverse_transform(y_pred_mean)
#    print(mpi_pred_mean_2GP,obs_train)
#    B = np.mean(mpi_pred_mean_2GP) / np.mean(obs_train)
#    y = (np.std(mpi_pred_mean_2GP) / np.mean(mpi_pred_mean_2GP)) / (np.std(obs_train) / np.mean(obs_train))
#    print(B,y)
#    mpi_pred_ECs_2GP[:,i] =np.squeeze(mpi_pred_mean_2GP)
#    print(mpi_pred_ECs_2GP[:,i])
#    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
#    y_pred_lb = y_pred_mean - y__pred_conf
#    y_pred_ub = y_pred_mean + y__pred_conf
#    mpi_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
#    mpi_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
mpi_hist_train_ECs_2GP_SGPRcor=tf.concat([mpi_pred_ECs_2GP[:,0].reshape(13149, 1),mpi_pred_ECs_2GP[:,1].reshape(13149, 1),mpi_pred_ECs_2GP[:,2].reshape(13149, 1),mpi_pred_ECs_2GP[:,3].reshape(13149, 1)],axis=1)
mpi_hist_train_ECs_2GP_lb_SGPRcor=tf.concat([mpi_pred_ECs_2GP_lb[:,0].reshape(13149, 1),mpi_pred_ECs_2GP_lb[:,1].reshape(13149, 1),mpi_pred_ECs_2GP_lb[:,2].reshape(13149, 1),mpi_pred_ECs_2GP_lb[:,3].reshape(13149, 1)],axis=1)
mpi_hist_train_ECs_2GP_ub_SGPRcor=tf.concat([mpi_pred_ECs_2GP_ub[:,0].reshape(13149, 1),mpi_pred_ECs_2GP_ub[:,1].reshape(13149, 1),mpi_pred_ECs_2GP_ub[:,2].reshape(13149, 1),mpi_pred_ECs_2GP_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(mpi_hist_train_ECs_2GP_SGPRcor,columns = ['mpi_EC1_2GP','mpi_EC2_2GP','mpi_EC3_2GP','mpi_EC4_2GP'])
df_lb_merge = pd.DataFrame(mpi_hist_train_ECs_2GP_lb_SGPRcor,columns = ['mpi_EC1_2GP_lb','mpi_EC2_2GP_lb','mpi_EC3_2GP_lb','mpi_EC4_2GP_lb'])
df_ub_merge = pd.DataFrame(mpi_hist_train_ECs_2GP_ub_SGPRcor,columns = ['mpi_EC1_2GP_ub','mpi_EC2_2GP_ub','mpi_EC3_2GP_ub','mpi_EC4_2GP_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean_2GP.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb_2GP.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub_2GP.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

In [10]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((13149,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [11]:
recondata_SGPR_mpi_mix4_2GP = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_hist_train_ECs_2GP_SGPRcor)

In [12]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(1979, 1, 1)
    end_date = datetime(2014, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 1979-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [13]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc = out_dir+'recondata_SGPR_mpi_mix4_2GP.nc' 
writeNetCDF(recondata_SGPR_mpi_mix4_2GP,out_file_nc) 

# 做未来预测

In [14]:
#读入数据
ssp126_mpi_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_mpi_ssp126.nc','r')
ssp370_mpi_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_mpi_ssp370.nc','r')
ssp585_mpi_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_mpi_ssp585.nc','r')

mpi_ssp126_train_origin = ssp126_mpi_simulation_data.variables['discharge'][:]
mpi_ssp370_train_origin = ssp370_mpi_simulation_data.variables['discharge'][:]
mpi_ssp585_train_origin = ssp585_mpi_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

mask_t = mpi_ssp126_train_origin == mpi_ssp126_train_origin[1][0][0]

mpi_ssp126_train = ma.array(mpi_ssp126_train_origin,mask = mask_t)
mpi_ssp370_train = ma.array(mpi_ssp370_train_origin,mask = mask_t)
mpi_ssp585_train = ma.array(mpi_ssp585_train_origin,mask = mask_t)

mpi_ssp126_train_ECs = obsdata_solver.projectField(mpi_ssp126_train,neofs = 4)
mpi_ssp370_train_ECs = obsdata_solver.projectField(mpi_ssp370_train,neofs = 4)
mpi_ssp585_train_ECs = obsdata_solver.projectField(mpi_ssp585_train,neofs = 4)

ssp126_mpi_simulation_data.close()
ssp370_mpi_simulation_data.close()
ssp585_mpi_simulation_data.close()

In [15]:
#将两列ECs各自做为训练集
mpi_ssp126_train_ECs_l1 = mpi_ssp126_train_ECs[:,0].reshape(28854,1)
mpi_ssp126_train_ECs_x1 = tf.convert_to_tensor(mpi_ssp126_train_ECs_l1)
mpi_ssp126_train_ECs_x1=tf.cast(mpi_ssp126_train_ECs_x1,tf.float64)
mpi_ssp126_train_ECs_l2 = mpi_ssp126_train_ECs[:,1].reshape(28854,1)
mpi_ssp126_train_ECs_x2 = tf.convert_to_tensor(mpi_ssp126_train_ECs_l2)
mpi_ssp126_train_ECs_x2=tf.cast(mpi_ssp126_train_ECs_x2,tf.float64)
mpi_ssp126_train_ECs_l3 = mpi_ssp126_train_ECs[:,2].reshape(28854,1)
mpi_ssp126_train_ECs_x3 = tf.convert_to_tensor(mpi_ssp126_train_ECs_l3)
mpi_ssp126_train_ECs_x3=tf.cast(mpi_ssp126_train_ECs_x3,tf.float64)
mpi_ssp126_train_ECs_l4 = mpi_ssp126_train_ECs[:,3].reshape(28854,1)
mpi_ssp126_train_ECs_x4 = tf.convert_to_tensor(mpi_ssp126_train_ECs_l4)
mpi_ssp126_train_ECs_x4=tf.cast(mpi_ssp126_train_ECs_x4,tf.float64)
mpi_ssp370_train_ECs_l1 = mpi_ssp370_train_ECs[:,0].reshape(28854,1)
mpi_ssp370_train_ECs_x1 = tf.convert_to_tensor(mpi_ssp370_train_ECs_l1)
mpi_ssp370_train_ECs_x1=tf.cast(mpi_ssp370_train_ECs_x1,tf.float64)
mpi_ssp370_train_ECs_l2 = mpi_ssp370_train_ECs[:,1].reshape(28854,1)
mpi_ssp370_train_ECs_x2 = tf.convert_to_tensor(mpi_ssp370_train_ECs_l2)
mpi_ssp370_train_ECs_x2=tf.cast(mpi_ssp370_train_ECs_x2,tf.float64)
mpi_ssp370_train_ECs_l3 = mpi_ssp370_train_ECs[:,2].reshape(28854,1)
mpi_ssp370_train_ECs_x3 = tf.convert_to_tensor(mpi_ssp370_train_ECs_l3)
mpi_ssp370_train_ECs_x3=tf.cast(mpi_ssp370_train_ECs_x3,tf.float64)
mpi_ssp370_train_ECs_l4 = mpi_ssp370_train_ECs[:,3].reshape(28854,1)
mpi_ssp370_train_ECs_x4 = tf.convert_to_tensor(mpi_ssp370_train_ECs_l4)
mpi_ssp370_train_ECs_x4=tf.cast(mpi_ssp370_train_ECs_x4,tf.float64)
mpi_ssp585_train_ECs_l1 = mpi_ssp585_train_ECs[:,0].reshape(28854,1)
mpi_ssp585_train_ECs_x1 = tf.convert_to_tensor(mpi_ssp585_train_ECs_l1)
mpi_ssp585_train_ECs_x1=tf.cast(mpi_ssp585_train_ECs_x1,tf.float64)
mpi_ssp585_train_ECs_l2 = mpi_ssp585_train_ECs[:,1].reshape(28854,1)
mpi_ssp585_train_ECs_x2 = tf.convert_to_tensor(mpi_ssp585_train_ECs_l2)
mpi_ssp585_train_ECs_x2=tf.cast(mpi_ssp585_train_ECs_x2,tf.float64)
mpi_ssp585_train_ECs_l3 = mpi_ssp585_train_ECs[:,2].reshape(28854,1)
mpi_ssp585_train_ECs_x3 = tf.convert_to_tensor(mpi_ssp585_train_ECs_l3)
mpi_ssp585_train_ECs_x3=tf.cast(mpi_ssp585_train_ECs_x3,tf.float64)
mpi_ssp585_train_ECs_l4 = mpi_ssp585_train_ECs[:,3].reshape(28854,1)
mpi_ssp585_train_ECs_x4 = tf.convert_to_tensor(mpi_ssp585_train_ECs_l4)
mpi_ssp585_train_ECs_x4=tf.cast(mpi_ssp585_train_ECs_x4,tf.float64)

In [16]:
mpi_pred_ECs_ssp126 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_lb_ssp126 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_ub_ssp126 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_ssp370 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_lb_ssp370 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_ub_ssp370 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_ssp585 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_lb_ssp585 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_ub_ssp585 = np.zeros((28854,n_input_ECs))
scaler_x_ssp126 = StandardScaler()
mpi_train_ssp126 = np.c_[mpi_ssp126_train_ECs_x1,mpi_ssp126_train_ECs_x2,mpi_ssp126_train_ECs_x3,mpi_ssp126_train_ECs_x4]
mpi_train_ssp126_sc = scaler_x_ssp126.fit_transform(mpi_train_ssp126)
scaler_x_ssp370 = StandardScaler()
mpi_train_ssp370 = np.c_[mpi_ssp370_train_ECs_x1,mpi_ssp370_train_ECs_x2,mpi_ssp370_train_ECs_x3,mpi_ssp370_train_ECs_x4]
mpi_train_ssp370_sc = scaler_x_ssp370.fit_transform(mpi_train_ssp370)
scaler_x_ssp585 = StandardScaler()
mpi_train_ssp585 = np.c_[mpi_ssp585_train_ECs_x1,mpi_ssp585_train_ECs_x2,mpi_ssp585_train_ECs_x3,mpi_ssp585_train_ECs_x4]
mpi_train_ssp585_sc = scaler_x_ssp585.fit_transform(mpi_train_ssp585)
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

In [17]:
#EOF1

for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mpi_train_sc[:,0].min(), mpi_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(mpi_train_sc[:, j].min(), mpi_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mpi_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mpi_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mpi_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_mpi_eof[i].training_loss, model_mpi_eof[i].trainable_variables)
    gf.utilities.print_summary(model_mpi_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_mpi_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_mpi_eof1_timeperiod:',opt_mpi_eof1_timeperiod)
    y_pred_mean_ssp126, y_pred_var_ssp126= model_mpi_eof[i].predict_y(mpi_train_ssp126_sc, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    mpi_pred_mean_ssp126 = scaler_y.inverse_transform(y_pred_mean_ssp126)
    mpi_pred_ECs_ssp126[:,i] =np.squeeze(mpi_pred_mean_ssp126)
    print(mpi_pred_ECs_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    mpi_pred_ECs_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    mpi_pred_ECs_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_ssp370, y_pred_var_ssp370= model_mpi_eof[i].predict_y(mpi_train_ssp370_sc, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    mpi_pred_mean_ssp370 = scaler_y.inverse_transform(y_pred_mean_ssp370)
    mpi_pred_ECs_ssp370[:,i] =np.squeeze(mpi_pred_mean_ssp370)
    print(mpi_pred_ECs_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    mpi_pred_ECs_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    mpi_pred_ECs_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_ssp585, y_pred_var_ssp585= model_mpi_eof[i].predict_y(mpi_train_ssp585_sc, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    mpi_pred_mean_ssp585 = scaler_y.inverse_transform(y_pred_mean_ssp585)
    mpi_pred_ECs_ssp585[:,i] =np.squeeze(mpi_pred_mean_ssp585)
    print(mpi_pred_ECs_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    mpi_pred_ECs_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    mpi_pred_ECs_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))
    y_pred_mean, y_pred_var = model_mpi_eof[i].predict_y(mpi_train_sc, full_cov=False)
    mpi_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mpi_pred_mean) / np.mean(obs_train)
    y = (np.std(mpi_pred_mean) / np.mean(mpi_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.99849
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.94257
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.21727
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.71359, -1.50457, -4.0545..."


The  1  ECs training for obsdata
opt_mpi_eof1_timeperiod: 4750.688000000024
tf.Tensor(
[[-0.01007846]
 [-0.02705682]
 [-0.29141019]
 ...
 [ 0.80116333]
 [ 0.84923386]
 [ 0.90019798]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-158258.77300558 -160622.18717678 -197420.59740808 ...  -45332.62284645
  -38641.12906557  -31546.84280243]
tf.Tensor(
[[ 0.09674188]
 [-0.08469944]
 [-0.31449469]
 ...
 [ 0.84240322]
 [ 0.84260915]
 [ 0.84287913]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-143389.21124381 -168646.13134165 -200633.99607263 ...  -39591.96521773
  -39563.29946581  -39525.71774169]
tf.Tensor(
[[ 0.15720463]
 [-0.13105665]
 [-0.18825439]
 ...
 [ 0.81203439]
 [ 0.81296422]
 [ 0.81348582]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-134972.7001

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.72238
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.65799
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.70799
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.77844, -1.77264, -4.18731..."


The  2  ECs training for obsdata
opt_mpi_eof1_timeperiod: 34.81200000003446
tf.Tensor(
[[ 1.16079196]
 [ 0.50330351]
 [ 0.06590411]
 ...
 [ 0.14053064]
 [ 0.04548237]
 [-0.07121217]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[19527.72773779  8255.50474488   756.56901832 ...  2035.99366067
   406.45125801 -1594.20264151]
tf.Tensor(
[[ 1.00040023]
 [ 0.66335744]
 [ 0.22667648]
 ...
 [-0.21202308]
 [-0.20671558]
 [-0.20274712]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[16777.91327176 10999.52805116  3512.90953024 ... -4008.31651159
 -3917.32283334 -3849.28608601]
tf.Tensor(
[[ 0.93347671]
 [ 0.84970953]
 [ 0.48853986]
 ...
 [-0.01674847]
 [-0.01251926]
 [-0.00819878]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[15630.55181107 141

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.31845
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.54808
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.86083
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.9063, -2.09145, -4.48771..."


The  3  ECs training for obsdata
opt_mpi_eof1_timeperiod: 14.625
tf.Tensor(
[[-0.3805646 ]
 [-0.84073582]
 [ 0.00259593]
 ...
 [ 0.07159401]
 [ 0.06154172]
 [ 0.08491023]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[-3554.9498717  -9309.10169097  1236.23154645 ...  2099.00915403
  1973.31159118  2265.52017324]
tf.Tensor(
[[-0.2770205 ]
 [-0.8788566 ]
 [ 0.02407862]
 ...
 [ 0.07766927]
 [ 0.07538343]
 [ 0.07378578]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[-2260.19601772 -9785.77800639  1504.85907565 ...  2174.97645504
  2146.39351871  2126.41588499]
tf.Tensor(
[[-0.22134873]
 [-0.78745308]
 [ 0.17515455]
 ...
 [ 0.0633998 ]
 [ 0.06395994]
 [ 0.06439963]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[-1564.05563615 -8642.83449502  3393.9686024  ..

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.64889
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.69431
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.71825
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.78636, -1.59575, -4.00971..."


The  4  ECs training for obsdata
opt_mpi_eof1_timeperiod: 33271.264999999956
tf.Tensor(
[[ 1.12487269]
 [-0.08786058]
 [-0.02421946]
 ...
 [ 0.02570068]
 [-0.11947327]
 [-0.29214493]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[11468.00848575  1132.27414124  1674.66694376 ...  2100.12016742
   862.84939661  -608.77541824]
tf.Tensor(
[[ 0.99507237]
 [ 0.28675228]
 [-0.05524863]
 ...
 [-0.28844817]
 [-0.28595306]
 [-0.28454357]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[10361.76224306  4324.97871211  1410.21534622 ...  -577.26913578
  -556.00410572  -543.99147822]
tf.Tensor(
[[ 0.94807497]
 [ 0.64224967]
 [-0.17253821]
 ...
 [-0.12430917]
 [-0.12247189]
 [-0.12010353]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[9961.21854642 735

In [18]:
#合并校正后的两个主成分-
mpi_ssp126_train_ECs_SGPRcor=tf.concat([mpi_pred_ECs_ssp126[:,0].reshape(28854, 1),mpi_pred_ECs_ssp126[:,1].reshape(28854, 1),mpi_pred_ECs_ssp126[:,2].reshape(28854, 1),mpi_pred_ECs_ssp126[:,3].reshape(28854, 1)],axis=1)
mpi_ssp126_train_ECs_lb_SGPRcor=tf.concat([mpi_pred_ECs_lb_ssp126[:,0].reshape(28854, 1),mpi_pred_ECs_lb_ssp126[:,1].reshape(28854, 1),mpi_pred_ECs_lb_ssp126[:,2].reshape(28854, 1),mpi_pred_ECs_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
mpi_ssp126_train_ECs_ub_SGPRcor=tf.concat([mpi_pred_ECs_ub_ssp126[:,0].reshape(28854, 1),mpi_pred_ECs_ub_ssp126[:,1].reshape(28854, 1),mpi_pred_ECs_ub_ssp126[:,2].reshape(28854, 1),mpi_pred_ECs_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
mpi_ssp370_train_ECs_SGPRcor=tf.concat([mpi_pred_ECs_ssp370[:,0].reshape(28854, 1),mpi_pred_ECs_ssp370[:,1].reshape(28854, 1),mpi_pred_ECs_ssp370[:,2].reshape(28854, 1),mpi_pred_ECs_ssp370[:,3].reshape(28854, 1)],axis=1)
mpi_ssp370_train_ECs_lb_SGPRcor=tf.concat([mpi_pred_ECs_lb_ssp370[:,0].reshape(28854, 1),mpi_pred_ECs_lb_ssp370[:,1].reshape(28854, 1),mpi_pred_ECs_lb_ssp370[:,2].reshape(28854, 1),mpi_pred_ECs_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
mpi_ssp370_train_ECs_ub_SGPRcor=tf.concat([mpi_pred_ECs_ub_ssp370[:,0].reshape(28854, 1),mpi_pred_ECs_ub_ssp370[:,1].reshape(28854, 1),mpi_pred_ECs_ub_ssp370[:,2].reshape(28854, 1),mpi_pred_ECs_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
mpi_ssp585_train_ECs_SGPRcor=tf.concat([mpi_pred_ECs_ssp585[:,0].reshape(28854, 1),mpi_pred_ECs_ssp585[:,1].reshape(28854, 1),mpi_pred_ECs_ssp585[:,2].reshape(28854, 1),mpi_pred_ECs_ssp585[:,3].reshape(28854, 1)],axis=1)
mpi_ssp585_train_ECs_lb_SGPRcor=tf.concat([mpi_pred_ECs_lb_ssp585[:,0].reshape(28854, 1),mpi_pred_ECs_lb_ssp585[:,1].reshape(28854, 1),mpi_pred_ECs_lb_ssp585[:,2].reshape(28854, 1),mpi_pred_ECs_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
mpi_ssp585_train_ECs_ub_SGPRcor=tf.concat([mpi_pred_ECs_ub_ssp585[:,0].reshape(28854, 1),mpi_pred_ECs_ub_ssp585[:,1].reshape(28854, 1),mpi_pred_ECs_ub_ssp585[:,2].reshape(28854, 1),mpi_pred_ECs_ub_ssp585[:,3].reshape(28854, 1)],axis=1)



In [20]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
mpi_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
mpi_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
mpi_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_mpi_eof_2GP = list()
mpi_pred_ECs_range = np.zeros((13149,n_input_ECs))
mpi_pred_ECs_2GP_ssp126 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_lb_ssp126 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_ub_ssp126 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_range_ssp126 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_ssp370 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_lb_ssp370 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_ub_ssp370 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_range_ssp370 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_ssp585 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_lb_ssp585 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_2GP_ub_ssp585 = np.zeros((28854,n_input_ECs))
mpi_pred_ECs_range_ssp585 = np.zeros((28854,n_input_ECs))
for i in range(n_input_ECs):
    mpi_pred_ECs_range[:,i] = np.subtract(mpi_pred_ECs_ub[:,i],mpi_pred_ECs_lb[:,i])
    mpi_train_2GP = np.c_[mpi_pred_ECs[:,i],mpi_pred_ECs_lb[:,i],mpi_pred_ECs_ub[:,i],mpi_pred_ECs_range[:,i] ]
    mpi_train_2GP_sc = scaler_z.fit_transform(mpi_train_2GP)
    mpi_pred_ECs_range_ssp126[:,i] = np.subtract(mpi_pred_ECs_ub_ssp126[:,i],mpi_pred_ECs_lb_ssp126[:,i])
    mpi_train_2GP_ssp126 = np.c_[mpi_pred_ECs_ssp126[:,i],mpi_pred_ECs_lb_ssp126[:,i],mpi_pred_ECs_ub_ssp126[:,i],mpi_pred_ECs_range_ssp126[:,i] ]
    mpi_train_2GP_sc_ssp126 = scaler_z.fit_transform(mpi_train_2GP_ssp126 )
    mpi_pred_ECs_range_ssp370[:,i] = np.subtract(mpi_pred_ECs_ub_ssp370[:,i],mpi_pred_ECs_lb_ssp370[:,i])
    mpi_train_2GP_ssp370 = np.c_[mpi_pred_ECs_ssp370[:,i],mpi_pred_ECs_lb_ssp370[:,i],mpi_pred_ECs_ub_ssp370[:,i],mpi_pred_ECs_range_ssp370[:,i] ]
    mpi_train_2GP_sc_ssp370 = scaler_z.fit_transform(mpi_train_2GP_ssp370 )
    mpi_pred_ECs_range_ssp585[:,i] = np.subtract(mpi_pred_ECs_ub_ssp585[:,i],mpi_pred_ECs_lb_ssp585[:,i])
    mpi_train_2GP_ssp585 = np.c_[mpi_pred_ECs_ssp585[:,i],mpi_pred_ECs_lb_ssp585[:,i],mpi_pred_ECs_ub_ssp585[:,i],mpi_pred_ECs_range_ssp585[:,i] ]
    mpi_train_2GP_sc_ssp585 = scaler_z.fit_transform(mpi_train_2GP_ssp585 )
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mpi_train_2GP_sc[:,0].min(), mpi_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(mpi_train_2GP_sc[:, j].min(), mpi_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mpi_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mpi_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mpi_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_mpi_eof_2GP[i].training_loss, model_mpi_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_mpi_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_mpi_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_mpi_eof1_2GP_timeperiod:',opt_mpi_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_mpi_eof_2GP[i].predict_y(mpi_train_2GP_sc, full_cov=False)
    mpi_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mpi_pred_mean) / np.mean(obs_train)
    y = (np.std(mpi_pred_mean) / np.mean(mpi_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    mpi_pred_ECs_2GP[:,i] =np.squeeze(mpi_pred_mean)
    print(mpi_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    mpi_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    mpi_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    y_pred_mean_2GP_ssp126, y_pred_var_2GP_ssp126= model_mpi_eof_2GP[i].predict_y(mpi_train_2GP_sc_ssp126, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    mpi_pred_mean_2GP_ssp126 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp126)
    mpi_pred_ECs_2GP_ssp126[:,i] =np.squeeze(mpi_pred_mean_2GP_ssp126)
    print(mpi_pred_ECs_2GP_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    mpi_pred_ECs_2GP_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    mpi_pred_ECs_2GP_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_2GP_ssp370, y_pred_var_2GP_ssp370= model_mpi_eof_2GP[i].predict_y(mpi_train_2GP_sc_ssp370, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    mpi_pred_mean_2GP_ssp370 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp370)
    mpi_pred_ECs_2GP_ssp370[:,i] =np.squeeze(mpi_pred_mean_2GP_ssp370)
    print(mpi_pred_ECs_2GP_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    mpi_pred_ECs_2GP_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    mpi_pred_ECs_2GP_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_2GP_ssp585, y_pred_var_2GP_ssp585= model_mpi_eof_2GP[i].predict_y(mpi_train_2GP_sc_ssp585, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    mpi_pred_mean_2GP_ssp585 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp585)
    mpi_pred_ECs_2GP_ssp585[:,i] =np.squeeze(mpi_pred_mean_2GP_ssp585)
    print(mpi_pred_ECs_2GP_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    mpi_pred_ECs_2GP_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    mpi_pred_ECs_2GP_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,195.62157
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,44.21345
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.20926
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-2.48975, -2.65501, -3.05686..."


The  1  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 33271.264999999956
[-42395.33177002 -37926.82855976 -34895.63608274 ... -31796.48693707
 -31856.40358536 -32038.2215125 ]
0.9999987469796829 0.889187902471384
tf.Tensor(
[[ 1.12487269]
 [-0.08786058]
 [-0.02421946]
 ...
 [ 0.02570068]
 [-0.11947327]
 [-0.29214493]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-155400.37487643 -152812.10831629 -217834.21032181 ...  -42712.71392167
  -36125.20219088  -29218.73138835]
tf.Tensor(
[[ 0.99507237]
 [ 0.28675228]
 [-0.05524863]
 ...
 [-0.28844817]
 [-0.28595306]
 [-0.28454357]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-137552.84038896 -162339.96274618 -199012.48165619 ...  -29531.49364096
  -29500.23214509  -29460.07308626]
tf.Tensor(
[[ 0.94807497]
 [ 0.64224967]
 [-0.17253821]
 ...
 [-0.12430917]
 [-0.12247189]
 [-0.120

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,2.79631
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,9.04003
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.66449
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.72115, -3.74807, -3.81078..."


The  2  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 33271.264999999956
[1458.81192241  276.87496725 -673.66777269 ...  -33.1400051    45.70902057
  142.19912479]
0.9995083470823839 0.5791796536657726
tf.Tensor(
[[ 1.12487269]
 [-0.08786058]
 [-0.02421946]
 ...
 [ 0.02570068]
 [-0.11947327]
 [-0.29214493]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[18192.912085   11883.65328074  -152.99750275 ...  2106.50584882
   363.18089532 -1827.72784876]
tf.Tensor(
[[ 0.99507237]
 [ 0.28675228]
 [-0.05524863]
 ...
 [-0.28844817]
 [-0.28595306]
 [-0.28454357]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[15541.40978559 13514.76495219  3763.60936338 ... -4849.54493709
 -4751.08687692 -4677.44813743]
tf.Tensor(
[[ 0.94807497]
 [ 0.64224967]
 [-0.17253821]
 ...
 [-0.12430917]
 [-0.12247189]
 [-0.12010353]], shape=(28854, 

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,27.21068
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,15.24717
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.80614
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.74421, -5.24104, -5.61034..."


The  3  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 33271.264999999956
[2416.51733076 2353.05914648 2381.33072187 ... 2353.78874974 2340.57561046
 2325.63634169]
1.0000202896610317 0.43983672469095914
tf.Tensor(
[[ 1.12487269]
 [-0.08786058]
 [-0.02421946]
 ...
 [ 0.02570068]
 [-0.11947327]
 [-0.29214493]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ -3583.80626575 -15373.18664426   2178.05624588 ...   2296.76411357
   2184.36728641   2493.91354055]
tf.Tensor(
[[ 0.99507237]
 [ 0.28675228]
 [-0.05524863]
 ...
 [-0.28844817]
 [-0.28595306]
 [-0.28454357]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ -1909.24702674 -14528.49981415   3385.03120758 ...   2725.29092402
   2695.86579477   2675.35671351]
tf.Tensor(
[[ 0.94807497]
 [ 0.64224967]
 [-0.17253821]
 ...
 [-0.12430917]
 [-0.12247189]
 [-0.12010353]], shape=(28854,

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.4358
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,1.49084
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.81948
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.32064, -4.32378, -4.31614..."


The  4  ECs training for obsdata
opt_mpi_eof1_2GP_timeperiod: 33271.264999999956
[1110.72527081  720.4143862   570.10988269 ...  381.32620262  381.10381292
  381.33161858]
1.0065562534800383 0.41905330580367595
tf.Tensor(
[[ 1.12487269]
 [-0.08786058]
 [-0.02421946]
 ...
 [ 0.02570068]
 [-0.11947327]
 [-0.29214493]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[ 6826.29411066 -2737.08207792  1002.03546042 ...  -934.93230739
 -2745.31110622 -3611.49087235]
tf.Tensor(
[[ 0.99507237]
 [ 0.28675228]
 [-0.05524863]
 ...
 [-0.28844817]
 [-0.28595306]
 [-0.28454357]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[ 6180.89412002  2582.97034678   408.03690596 ... -3796.597197
 -3798.60022383 -3798.78630004]
tf.Tensor(
[[ 0.94807497]
 [ 0.64224967]
 [-0.17253821]
 ...
 [-0.12430917]
 [-0.12247189]
 [-0.12010353]], shape=(28854, 1

In [21]:
#合并校正后的两个主成分-
mpi_ssp126_train_ECs_2GP_SGPRcor=tf.concat([mpi_pred_ECs_2GP_ssp126[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_ssp126[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_ssp126[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_ssp126[:,3].reshape(28854, 1)],axis=1)
mpi_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([mpi_pred_ECs_2GP_lb_ssp126[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp126[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp126[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
mpi_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([mpi_pred_ECs_2GP_ub_ssp126[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp126[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp126[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
mpi_ssp370_train_ECs_2GP_SGPRcor=tf.concat([mpi_pred_ECs_2GP_ssp370[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_ssp370[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_ssp370[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_ssp370[:,3].reshape(28854, 1)],axis=1)
mpi_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([mpi_pred_ECs_2GP_lb_ssp370[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp370[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp370[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
mpi_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([mpi_pred_ECs_2GP_ub_ssp370[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp370[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp370[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
mpi_ssp585_train_ECs_2GP_SGPRcor=tf.concat([mpi_pred_ECs_2GP_ssp585[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_ssp585[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_ssp585[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_ssp585[:,3].reshape(28854, 1)],axis=1)
mpi_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([mpi_pred_ECs_2GP_lb_ssp585[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp585[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp585[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
mpi_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([mpi_pred_ECs_2GP_ub_ssp585[:,0].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp585[:,1].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp585[:,2].reshape(28854, 1),mpi_pred_ECs_2GP_ub_ssp585[:,3].reshape(28854, 1)],axis=1)

# 写入Excel文件
df_mean_merge_2GP_ssp126 = pd.DataFrame(mpi_ssp126_train_ECs_2GP_SGPRcor,columns = ['mpi_EC1_2GP','mpi_EC2_2GP','mpi_EC3_2GP','mpi_EC4_2GP'])
df_lb_merge_2GP_ssp126 = pd.DataFrame(mpi_ssp126_train_ECs_2GP_lb_SGPRcor,columns = ['mpi_EC1_2GP_lb','mpi_EC2_2GP_lb','mpi_EC3_2GP_lb','mpi_EC4_2GP_lb'])
df_ub_merge_2GP_ssp126 = pd.DataFrame(mpi_ssp126_train_ECs_2GP_ub_SGPRcor,columns = ['mpi_EC1_2GP_ub','mpi_EC2_2GP_ub','mpi_EC3_2GP_ub','mpi_EC4_2GP_ub'])
df_mean_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean_ssp126_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb_ssp126_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub_ssp126_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp370 = pd.DataFrame(mpi_ssp370_train_ECs_2GP_SGPRcor,columns = ['mpi_EC1_2GP','mpi_EC2_2GP','mpi_EC3_2GP','mpi_EC4_2GP'])
df_lb_merge_2GP_ssp370 = pd.DataFrame(mpi_ssp370_train_ECs_2GP_lb_SGPRcor,columns = ['mpi_EC1_2GP_lb','mpi_EC2_2GP_lb','mpi_EC3_2GP_lb','mpi_EC4_2GP_lb'])
df_ub_merge_2GP_ssp370 = pd.DataFrame(mpi_ssp370_train_ECs_2GP_ub_SGPRcor,columns = ['mpi_EC1_2GP_ub','mpi_EC2_2GP_ub','mpi_EC3_2GP_ub','mpi_EC4_2GP_ub'])
df_mean_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean_ssp370_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb_ssp370_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub_ssp370_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp585 = pd.DataFrame(mpi_ssp585_train_ECs_2GP_SGPRcor,columns = ['mpi_EC1_2GP','mpi_EC2_2GP','mpi_EC3_2GP','mpi_EC4_2GP'])
df_lb_merge_2GP_ssp585 = pd.DataFrame(mpi_ssp585_train_ECs_2GP_lb_SGPRcor,columns = ['mpi_EC1_2GP_lb','mpi_EC2_2GP_lb','mpi_EC3_2GP_lb','mpi_EC4_2GP_lb'])
df_ub_merge_2GP_ssp585 = pd.DataFrame(mpi_ssp585_train_ECs_2GP_ub_SGPRcor,columns = ['mpi_EC1_2GP_ub','mpi_EC2_2GP_ub','mpi_EC3_2GP_ub','mpi_EC4_2GP_ub'])
df_mean_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean_ssp585_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb_ssp585_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub_ssp585_2GP.xlsx',index=False)

In [23]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean_ssp126_2GP.xlsx')
# 获取指定列的数据
mpi_ssp126_train_SGPRcor_EC1_2GP = data_frame['mpi_EC1_2GP']
mpi_ssp126_train_SGPRcor_EC1_2GP_array = np.array(mpi_ssp126_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC1_2GP_array)
mpi_ssp126_train_SGPRcor_EC1_2GP_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC1_2GP_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC2_2GP = data_frame['mpi_EC2_2GP']
mpi_ssp126_train_SGPRcor_EC2_2GP_array = np.array(mpi_ssp126_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC2_2GP_array)
mpi_ssp126_train_SGPRcor_EC2_2GP_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC2_2GP_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC3_2GP = data_frame['mpi_EC3_2GP']
mpi_ssp126_train_SGPRcor_EC3_2GP_array = np.array(mpi_ssp126_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC3_2GP_array)
mpi_ssp126_train_SGPRcor_EC3_2GP_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC3_2GP_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC4_2GP = data_frame['mpi_EC4_2GP']
mpi_ssp126_train_SGPRcor_EC4_2GP_array = np.array(mpi_ssp126_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC4_2GP_array)
mpi_ssp126_train_SGPRcor_EC4_2GP_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC4_2GP_tf,tf.float64)
mpi_ssp126_train_ECs_2GP_SGPRcor=tf.concat([mpi_ssp126_train_SGPRcor_EC1_2GP_tf,mpi_ssp126_train_SGPRcor_EC2_2GP_tf,mpi_ssp126_train_SGPRcor_EC3_2GP_tf,mpi_ssp126_train_SGPRcor_EC4_2GP_tf],axis=1)
print(mpi_ssp126_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean_ssp370_2GP.xlsx')
# 获取指定列的数据
mpi_ssp370_train_SGPRcor_EC1_2GP = data_frame['mpi_EC1_2GP']
mpi_ssp370_train_SGPRcor_EC1_2GP_array = np.array(mpi_ssp370_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC1_2GP_array)
mpi_ssp370_train_SGPRcor_EC1_2GP_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC1_2GP_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC2_2GP = data_frame['mpi_EC2_2GP']
mpi_ssp370_train_SGPRcor_EC2_2GP_array = np.array(mpi_ssp370_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC2_2GP_array)
mpi_ssp370_train_SGPRcor_EC2_2GP_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC2_2GP_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC3_2GP = data_frame['mpi_EC3_2GP']
mpi_ssp370_train_SGPRcor_EC3_2GP_array = np.array(mpi_ssp370_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC3_2GP_array)
mpi_ssp370_train_SGPRcor_EC3_2GP_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC3_2GP_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC4_2GP = data_frame['mpi_EC4_2GP']
mpi_ssp370_train_SGPRcor_EC4_2GP_array = np.array(mpi_ssp370_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC4_2GP_array)
mpi_ssp370_train_SGPRcor_EC4_2GP_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC4_2GP_tf,tf.float64)
mpi_ssp370_train_ECs_2GP_SGPRcor=tf.concat([mpi_ssp370_train_SGPRcor_EC1_2GP_tf,mpi_ssp370_train_SGPRcor_EC2_2GP_tf,mpi_ssp370_train_SGPRcor_EC3_2GP_tf,mpi_ssp370_train_SGPRcor_EC4_2GP_tf],axis=1)
print(mpi_ssp370_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_mean_ssp585_2GP.xlsx')
# 获取指定列的数据
mpi_ssp585_train_SGPRcor_EC1_2GP = data_frame['mpi_EC1_2GP']
mpi_ssp585_train_SGPRcor_EC1_2GP_array = np.array(mpi_ssp585_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC1_2GP_array)
mpi_ssp585_train_SGPRcor_EC1_2GP_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC1_2GP_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC2_2GP = data_frame['mpi_EC2_2GP']
mpi_ssp585_train_SGPRcor_EC2_2GP_array = np.array(mpi_ssp585_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC2_2GP_array)
mpi_ssp585_train_SGPRcor_EC2_2GP_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC2_2GP_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC3_2GP = data_frame['mpi_EC3_2GP']
mpi_ssp585_train_SGPRcor_EC3_2GP_array = np.array(mpi_ssp585_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC3_2GP_array)
mpi_ssp585_train_SGPRcor_EC3_2GP_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC3_2GP_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC4_2GP = data_frame['mpi_EC4_2GP']
mpi_ssp585_train_SGPRcor_EC4_2GP_array = np.array(mpi_ssp585_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC4_2GP_array)
mpi_ssp585_train_SGPRcor_EC4_2GP_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC4_2GP_tf,tf.float64)
mpi_ssp585_train_ECs_2GP_SGPRcor=tf.concat([mpi_ssp585_train_SGPRcor_EC1_2GP_tf,mpi_ssp585_train_SGPRcor_EC2_2GP_tf,mpi_ssp585_train_SGPRcor_EC3_2GP_tf,mpi_ssp585_train_SGPRcor_EC4_2GP_tf],axis=1)
print(mpi_ssp585_train_ECs_2GP_SGPRcor)

tf.Tensor(
[[-1.55400375e+05  1.81929121e+04 -3.58380627e+03  6.82629411e+03]
 [-1.52812108e+05  1.18836533e+04 -1.53731866e+04 -2.73708208e+03]
 [-2.17834210e+05 -1.52997503e+02  2.17805625e+03  1.00203546e+03]
 ...
 [-4.27127139e+04  2.10650585e+03  2.29676411e+03 -9.34932307e+02]
 [-3.61252022e+04  3.63180895e+02  2.18436729e+03 -2.74531111e+03]
 [-2.92187314e+04 -1.82772785e+03  2.49391354e+03 -3.61149087e+03]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-137552.84038896   15541.40978559   -1909.24702674    6180.89412002]
 [-162339.96274618   13514.76495219  -14528.49981415    2582.97034678]
 [-199012.48165619    3763.60936338    3385.03120758     408.03690596]
 ...
 [ -29531.49364096   -4849.54493709    2725.29092402   -3796.597197  ]
 [ -29500.23214509   -4751.08687692    2695.86579477   -3798.60022383]
 [ -29460.07308626   -4677.44813743    2675.35671351   -3798.78630004]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-128076.42770173   14643.62464069   -1286.95892505    621

In [24]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb_ssp126_2GP.xlsx')
# 获取指定列的数据
mpi_ssp126_train_SGPRcor_EC1_2GP_lb = data_frame['mpi_EC1_2GP_lb']
mpi_ssp126_train_SGPRcor_EC1_2GP_lb_array = np.array(mpi_ssp126_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC1_2GP_lb_array)
mpi_ssp126_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC2_2GP_lb = data_frame['mpi_EC2_2GP_lb']
mpi_ssp126_train_SGPRcor_EC2_2GP_lb_array = np.array(mpi_ssp126_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC2_2GP_lb_array)
mpi_ssp126_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC3_2GP_lb = data_frame['mpi_EC3_2GP_lb']
mpi_ssp126_train_SGPRcor_EC3_2GP_lb_array = np.array(mpi_ssp126_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC3_2GP_lb_array)
mpi_ssp126_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC4_2GP_lb = data_frame['mpi_EC4_2GP_lb']
mpi_ssp126_train_SGPRcor_EC4_2GP_lb_array = np.array(mpi_ssp126_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC4_2GP_lb_array)
mpi_ssp126_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
mpi_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([mpi_ssp126_train_SGPRcor_EC1_2GP_lb_tf,mpi_ssp126_train_SGPRcor_EC2_2GP_lb_tf,mpi_ssp126_train_SGPRcor_EC3_2GP_lb_tf,mpi_ssp126_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(mpi_ssp126_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb_ssp370_2GP.xlsx')
# 获取指定列的数据
mpi_ssp370_train_SGPRcor_EC1_2GP_lb = data_frame['mpi_EC1_2GP_lb']
mpi_ssp370_train_SGPRcor_EC1_2GP_lb_array = np.array(mpi_ssp370_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC1_2GP_lb_array)
mpi_ssp370_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC2_2GP_lb = data_frame['mpi_EC2_2GP_lb']
mpi_ssp370_train_SGPRcor_EC2_2GP_lb_array = np.array(mpi_ssp370_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC2_2GP_lb_array)
mpi_ssp370_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC3_2GP_lb = data_frame['mpi_EC3_2GP_lb']
mpi_ssp370_train_SGPRcor_EC3_2GP_lb_array = np.array(mpi_ssp370_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC3_2GP_lb_array)
mpi_ssp370_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC4_2GP_lb = data_frame['mpi_EC4_2GP_lb']
mpi_ssp370_train_SGPRcor_EC4_2GP_lb_array = np.array(mpi_ssp370_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC4_2GP_lb_array)
mpi_ssp370_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
mpi_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([mpi_ssp370_train_SGPRcor_EC1_2GP_lb_tf,mpi_ssp370_train_SGPRcor_EC2_2GP_lb_tf,mpi_ssp370_train_SGPRcor_EC3_2GP_lb_tf,mpi_ssp370_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(mpi_ssp370_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_lb_ssp585_2GP.xlsx')
# 获取指定列的数据
mpi_ssp585_train_SGPRcor_EC1_2GP_lb = data_frame['mpi_EC1_2GP_lb']
mpi_ssp585_train_SGPRcor_EC1_2GP_lb_array = np.array(mpi_ssp585_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC1_2GP_lb_array)
mpi_ssp585_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC2_2GP_lb = data_frame['mpi_EC2_2GP_lb']
mpi_ssp585_train_SGPRcor_EC2_2GP_lb_array = np.array(mpi_ssp585_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC2_2GP_lb_array)
mpi_ssp585_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC3_2GP_lb = data_frame['mpi_EC3_2GP_lb']
mpi_ssp585_train_SGPRcor_EC3_2GP_lb_array = np.array(mpi_ssp585_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC3_2GP_lb_array)
mpi_ssp585_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC4_2GP_lb = data_frame['mpi_EC4_2GP_lb']
mpi_ssp585_train_SGPRcor_EC4_2GP_lb_array = np.array(mpi_ssp585_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC4_2GP_lb_array)
mpi_ssp585_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
mpi_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([mpi_ssp585_train_SGPRcor_EC1_2GP_lb_tf,mpi_ssp585_train_SGPRcor_EC2_2GP_lb_tf,mpi_ssp585_train_SGPRcor_EC3_2GP_lb_tf,mpi_ssp585_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(mpi_ssp585_train_ECs_2GP_lb_SGPRcor)

tf.Tensor(
[[-232434.62084418   -9681.76046122   -5585.41266515   -2746.26207816]
 [-428075.54091596  -33777.31564556  -23159.68485613  -14724.46141722]
 [-478990.29960608  -40048.08649077  -27733.31816061  -17841.73938067]
 ...
 [-385146.67426854  -28490.10438667  -19303.41804326  -12096.12312319]
 [-405601.78969177  -31009.40025944  -21140.88520511  -13348.49634835]
 [-429608.29423675  -33966.09283088  -23297.37089876  -14818.30489744]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-250222.31917416  -11872.53152565   -7183.26781727   -3835.3215464 ]
 [-364892.58502375  -25995.56732904  -17484.00890719  -10856.05781213]
 [-481498.31916378  -40356.97955687  -27958.61161328  -17995.29395503]
 ...
 [-428369.76902286  -33813.55341005  -23186.11513889  -14742.4756594 ]
 [-428028.63568524  -33771.53869678  -23155.47139562  -14721.58962432]
 [-427837.77800206  -33748.03225531  -23138.32679789  -14709.90428072]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-256718.86496892  -12672.6600348 

In [7]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub_ssp126_2GP.xlsx')
# 获取指定列的数据
mpi_ssp126_train_SGPRcor_EC1_2GP_ub = data_frame['mpi_EC1_2GP_ub']
mpi_ssp126_train_SGPRcor_EC1_2GP_ub_array = np.array(mpi_ssp126_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC1_2GP_ub_array)
mpi_ssp126_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC2_2GP_ub = data_frame['mpi_EC2_2GP_ub']
mpi_ssp126_train_SGPRcor_EC2_2GP_ub_array = np.array(mpi_ssp126_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC2_2GP_ub_array)
mpi_ssp126_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC3_2GP_ub = data_frame['mpi_EC3_2GP_ub']
mpi_ssp126_train_SGPRcor_EC3_2GP_ub_array = np.array(mpi_ssp126_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC3_2GP_ub_array)
mpi_ssp126_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
mpi_ssp126_train_SGPRcor_EC4_2GP_ub = data_frame['mpi_EC4_2GP_ub']
mpi_ssp126_train_SGPRcor_EC4_2GP_ub_array = np.array(mpi_ssp126_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp126_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp126_train_SGPRcor_EC4_2GP_ub_array)
mpi_ssp126_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(mpi_ssp126_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
mpi_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([mpi_ssp126_train_SGPRcor_EC1_2GP_ub_tf,mpi_ssp126_train_SGPRcor_EC2_2GP_ub_tf,mpi_ssp126_train_SGPRcor_EC3_2GP_ub_tf,mpi_ssp126_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(mpi_ssp126_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub_ssp370_2GP.xlsx')
# 获取指定列的数据
mpi_ssp370_train_SGPRcor_EC1_2GP_ub = data_frame['mpi_EC1_2GP_ub']
mpi_ssp370_train_SGPRcor_EC1_2GP_ub_array = np.array(mpi_ssp370_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC1_2GP_ub_array)
mpi_ssp370_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC2_2GP_ub = data_frame['mpi_EC2_2GP_ub']
mpi_ssp370_train_SGPRcor_EC2_2GP_ub_array = np.array(mpi_ssp370_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC2_2GP_ub_array)
mpi_ssp370_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC3_2GP_ub = data_frame['mpi_EC3_2GP_ub']
mpi_ssp370_train_SGPRcor_EC3_2GP_ub_array = np.array(mpi_ssp370_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC3_2GP_ub_array)
mpi_ssp370_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
mpi_ssp370_train_SGPRcor_EC4_2GP_ub = data_frame['mpi_EC4_2GP_ub']
mpi_ssp370_train_SGPRcor_EC4_2GP_ub_array = np.array(mpi_ssp370_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp370_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp370_train_SGPRcor_EC4_2GP_ub_array)
mpi_ssp370_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(mpi_ssp370_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
mpi_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([mpi_ssp370_train_SGPRcor_EC1_2GP_ub_tf,mpi_ssp370_train_SGPRcor_EC2_2GP_ub_tf,mpi_ssp370_train_SGPRcor_EC3_2GP_ub_tf,mpi_ssp370_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(mpi_ssp370_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mpi_ECs_pred_ub_ssp585_2GP.xlsx')
# 获取指定列的数据
mpi_ssp585_train_SGPRcor_EC1_2GP_ub = data_frame['mpi_EC1_2GP_ub']
mpi_ssp585_train_SGPRcor_EC1_2GP_ub_array = np.array(mpi_ssp585_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC1_2GP_ub_array)
mpi_ssp585_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC2_2GP_ub = data_frame['mpi_EC2_2GP_ub']
mpi_ssp585_train_SGPRcor_EC2_2GP_ub_array = np.array(mpi_ssp585_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC2_2GP_ub_array)
mpi_ssp585_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC3_2GP_ub = data_frame['mpi_EC3_2GP_ub']
mpi_ssp585_train_SGPRcor_EC3_2GP_ub_array = np.array(mpi_ssp585_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC3_2GP_ub_array)
mpi_ssp585_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
mpi_ssp585_train_SGPRcor_EC4_2GP_ub = data_frame['mpi_EC4_2GP_ub']
mpi_ssp585_train_SGPRcor_EC4_2GP_ub_array = np.array(mpi_ssp585_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
mpi_ssp585_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(mpi_ssp585_train_SGPRcor_EC4_2GP_ub_array)
mpi_ssp585_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(mpi_ssp585_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
mpi_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([mpi_ssp585_train_SGPRcor_EC1_2GP_ub_tf,mpi_ssp585_train_SGPRcor_EC2_2GP_ub_tf,mpi_ssp585_train_SGPRcor_EC3_2GP_ub_tf,mpi_ssp585_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(mpi_ssp585_train_ECs_2GP_ub_SGPRcor)

tf.Tensor(
[[231891.09165309  47505.58983792  36124.6061474   25682.27904965]
 [ 89903.20466633  30018.05724431  23369.9443271   16989.00969969]
 [158535.8563255   38471.00177603  29535.16222224  21191.07326819]
 ...
 [ 78590.15633854  28624.71795627  22353.70196559  16296.36345803]
 [ 58628.38018628  26166.18280586  20560.55108802  15074.19514157]
 [ 34562.54520373  23202.18297051  18398.73578715  13600.75406096]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[213541.96730713  45245.67234162  34476.31853148  24558.84603251]
 [131013.66418348  35081.30958225  27062.86503707  19506.01523634]
 [152405.25627224  37715.94392902  28984.45517184  20815.72464747]
 ...
 [ 34353.20942199  23176.4007268   18379.93131615  13587.93738784]
 [ 34706.72307246  23219.94022595  18411.68717334  13609.58141289]
 [ 34908.27186526  23244.76340741  18429.79214515  13621.92132427]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[206954.28647972  44434.31944645  33884.55226946  24155.51244512]
 [176133.6843075

In [8]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((28854,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [9]:
#recondata_SGPR_mpi_mix4_2GP_ssp126 = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp126_train_ECs_2GP_SGPRcor)
#recondata_SGPR_mpi_mix4_2GP_ssp370 = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp370_train_ECs_2GP_SGPRcor)
#recondata_SGPR_mpi_mix4_2GP_ssp585 = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp585_train_ECs_2GP_SGPRcor)
#recondata_SGPR_mpi_mix4_2GP_ssp126_lb = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp126_train_ECs_2GP_lb_SGPRcor)
#recondata_SGPR_mpi_mix4_2GP_ssp370_lb = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp370_train_ECs_2GP_lb_SGPRcor)
#recondata_SGPR_mpi_mix4_2GP_ssp585_lb = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp585_train_ECs_2GP_lb_SGPRcor)
#recondata_SGPR_mpi_mix4_2GP_ssp126_ub = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp126_train_ECs_2GP_ub_SGPRcor)
#recondata_SGPR_mpi_mix4_2GP_ssp370_ub = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp370_train_ECs_2GP_ub_SGPRcor)
recondata_SGPR_mpi_mix4_2GP_ssp585_ub = ReconstructDataFromEOFandECs(obsdata_eofs,mpi_ssp585_train_ECs_2GP_ub_SGPRcor)

In [10]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2100, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 2022-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [11]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc1 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp126.nc' 
out_file_nc2 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp370.nc' 
out_file_nc3 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp585.nc'
out_file_nc4 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp126_lb.nc' 
out_file_nc5 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp370_lb.nc' 
out_file_nc6 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp585_lb.nc' 
out_file_nc7 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp126_ub.nc' 
out_file_nc8 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp370_ub.nc' 
out_file_nc9 = out_dir+'recondata_SGPR_mpi_mix4_2GP_ssp585_ub.nc' 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp126,out_file_nc1) 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp370,out_file_nc2) 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp585,out_file_nc3) 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp126_lb,out_file_nc4) 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp370_lb,out_file_nc5) 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp585_lb,out_file_nc6) 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp126_ub,out_file_nc7) 
#writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp370_ub,out_file_nc8) 
writeNetCDF(recondata_SGPR_mpi_mix4_2GP_ssp585_ub,out_file_nc9) 